A web scraper is used to extract data from websites which can further be used to perform analysis. This python script is ot be used specifically for extracting data from Amazon.in

PS: price of the products are in INR

Follow the steps to generate a dataset of your own:

1.Initialize search_term to whatever term you are using to scrape the data.
2.Initialize csv_file to the target csv file where you want to store the data. The csv file need not be created manaually, just a name for the file is sufficient
3.Run the script, and you get the dataset for your search term.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import time
import datetime

import requests

import csv

In [2]:
search_term='watches for men'

csv_file='amazonlarge.csv'  

### go to https://httpbin.org/get
### copy contents of User-Agent and paste it in user_agent
user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"

###do not change attributes
attributes = ['Brand','Title','Price', 'Ratings','Reviews','Date of Extraction']
    

In [3]:
def get_url(search_term):
    
    parent='https://www.amazon.in/s?k={}'
    search_term=search_term.replace(' ','+')
    
    url=parent.format(search_term)
    
    url=url+'&page={}'
    
    return url

In [4]:
def add_attributes(attributes,csv_file):
    
    with open(csv_file,'w',encoding='UTF8',newline='') as f:
        writer=csv.writer(f)
        writer.writerow(attributes)
        
        

In [5]:
def add_data(url,headers,csv_file):
    
    page=requests.get(url,headers=headers)

    soup1=BeautifulSoup(page.content,'html.parser')
    soup2=BeautifulSoup(soup1.prettify(),'html.parser')
    
    lists=soup2.find_all('div',class_='a-section a-spacing-base a-text-center')
    
    today = datetime.date.today()
    
    with open(csv_file,'a+',encoding='UTF8',newline='') as f:
        writer=csv.writer(f)
        
        for items in lists:
            try:
                brand=items.find('span',class_='a-size-base-plus a-color-base').text.strip()
                title=items.find('span',class_='a-size-base-plus a-color-base a-text-normal').text.replace('\n','')
                price=items.find('span',class_='a-price').text.strip()
                price=price.strip()[1:15]
                price=price.strip()
            except AttributeError:
                pass

            try:
                rating=items.find('span',class_='a-icon-alt').text.strip()
                review=items.find('span',class_='a-size-base s-underline-text').text.strip()
            except AttributeError:
                rating=""
                review=""
            data=[brand,title,price,rating,review,today]
            writer.writerow(data)
    

In [6]:
headers={"User-Agent": user_agent,
         "Accept-Encoding":"gzip, deflate", 
         "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
         "DNT":"1","Connection":"close",
         "Upgrade-Insecure-Requests":"1"}

add_attributes(attributes,csv_file)

for i in range(1,21):   #max 20 pages in amazon.in
    url=get_url(search_term)
    url=url.format(i)
    if url:
        add_data(url,headers,csv_file)

In [7]:
df=pd.read_csv(csv_file)

In [11]:
df

,Brand,Title,Price,Ratings,Reviews,Date of Extraction
0,Tommy Hilfiger,Analog Blue Dial Men's Wa...,"14,250",3.9 out of 5 stars,24,2022-09-18
1,Tommy Hilfiger,Analog Black Dial Men's W...,"14,500",4.4 out of 5 stars,16,2022-09-18
2,TIMEX,Analog Men's Watch ...,999,4.2 out of 5 stars,"16,504",2022-09-18
3,Tommy Hilfiger,Analog Blue Dial Men's Wa...,"10,495",4.5 out of 5 stars,26,2022-09-18
4,Matrix,Day & Date Display Analog W...,299,3.7 out of 5 stars,"1,481",2022-09-18
...,...,...,...,...,...,...
545,V2A,Digital Men's Watch (Blac...,560,3.8 out of 5 stars,166,2022-09-18
546,TIMEX,Analog Men's Watch ...,999,4.2 out of 5 stars,"16,504",2022-09-18
547,Titan,Neo Analog Grey Dial Men'...,"4,737",4.3 out of 5 stars,157,2022-09-18
548,Titan,Analog Dial Men's Watch ...,"2,875",4.3 out of 5 stars,561,2022-09-18
